In [1]:
from web3 import Web3
from hexbytes import HexBytes
import pandas as pd
import numpy as np
import rlp
import requests
from EtherScan import EtherScan
from web3 import Web3
from dotenv import load_dotenv

from datetime import datetime, timedelta, timezone

import matplotlib.pylab as plt
import numpy as np
import json
import logging
import typing
import requests
import sys
import time

import pandas as pd
import seaborn as sns
from dotenv import load_dotenv
from web3 import Web3

from EtherScan import EtherScan
from FortaExplorer import FortaExplorer
from Plotter import Plotter

load_dotenv()

True

In [2]:

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [3]:
def empty_labels() -> pd.DataFrame:
        df_forta = pd.DataFrame(columns=['createdAt', 'id', 'label', 'source'])
        return df_forta

def labels_by_source_id(source_id: str, start_date: datetime, end_date: datetime) -> pd.DataFrame:
    url = "https://api.forta.network/graphql"
    chunk_size = 8000

    df_forta = empty_labels()
    json_data = ""
    first_run = True
    count = 0
    while (json_data == "" or json_data['data']['labels']['pageInfo']['hasNextPage']):
        query = """query exampleQuery {
                    labels(
                        input: {
                            SOURCEIDS_CLAUSE
                            CREATEDBEFORE_CLAUSE
                            CREATEDSINCE_CLAUSE
                            AFTER_CLAUSE
                            CHUNKSIZE
                            state: true
                        }
                    ) {
                        pageInfo {
                            endCursor {
                                pageToken
                            }
                            hasNextPage
                        }
                        labels {
                            createdAt
                            id
                            label {
                                label
                                metadata
                                remove
                                entityType
                                entity
                                confidence
                            }
                            source {
                                chainId
                                
                                alertId
                            }
                        }
                    }
                }"""

        after_clause = ""
        if(first_run is False):
            pageToken = json_data['data']['labels']['pageInfo']['endCursor']['pageToken']
            after_clause = """after: {{pageToken:"{0}"}}""".format(pageToken)

        # this is a bit hacky
        query = query.replace("SOURCEIDS_CLAUSE", f"""sourceIds: ["{source_id}"]""")
        query = query.replace("CREATEDBEFORE_CLAUSE", f"""createdBefore: {int(end_date.timestamp()*1000)}""")
        query = query.replace("CREATEDSINCE_CLAUSE", f"""createdSince: {int(start_date.timestamp()*1000)}""")
        query = query.replace("AFTER_CLAUSE", after_clause)
        query = query.replace("CHUNKSIZE", f"first: {chunk_size}") 

        #print(query)

        retries = 1
        wait = 1
        success = False
        while not success:
            try:
                count += 1
                r = requests.post(url, json={'query': query})
                if r.status_code == 200:
                    success = True
                    if chunk_size < 5000:
                        chunk_size *= 2
                        logging.warning(f"Increasing chunk size to {chunk_size}")
                else:
                    raise Exception(f"status code: {r.status_code} {r.text}")
            except Exception as e:
                logging.warning(f"Unable to retrieve alerts {r.status_code} , {e}")
                logging.warning(f"Sleeping {wait}sec. Count {count}.")
                old_chunk_size = chunk_size
                chunk_size = int(chunk_size/2)
                if(chunk_size<1):
                    chunk_size = 1
                query = query.replace(f"first: {old_chunk_size},", f"first: {chunk_size},") 
                logging.warning(f"Reducing chunk size to {chunk_size}")
                time.sleep(wait)
                retries += 1
                if retries > 30:
                    raise Exception("Unable to retrieve alerts even after repeated retries. Pls check logs")

        json_data = json.loads(r.text)
        df_data = json_data['data']['labels']['labels']
        df_forta = pd.concat([pd.DataFrame(df_data), df_forta])

        first_run = False
        count += 1

    return df_forta




In [4]:
start_date = datetime(2023, 6, 14, 0, 0, 0, 0, timezone.utc)
end_date = datetime(2023, 6, 22, 0, 0, 0, 0, timezone.utc)

In [5]:
source_id = "0x47c45816807d2eac30ba88745bf2778b61bc106bc76411b520a5289495c76db8"

file_name = f"ScamDetector_{source_id}_labels_{start_date.strftime('%Y%m%d')}-{end_date.strftime('%Y%m%d')}.csv"

df_forta_labels = empty_labels() # 'createdAt', 'id', 'label', 'source'
df_forta_labels = labels_by_source_id(source_id, start_date, end_date)


# for index, row in df_forta_labels.iterrows():
#     created_date = row['createdAt']
#     source = row['source']
#     alert_id = source['alertId']
#     label = row['label']
#     entity = label['entity']
#     entityType = label['entityType']
#     label_tag = label['label']
#     remove = label['remove']
#     confidence = label['confidence']
#     metadata = label['metadata']
#     handler_type = ""
#     if 'handlerType' in metadata:
#         handler_type = metadata['handlerType']
#     chain_id = source['chainId']
#     df_temp_labels = pd.DataFrame(columns=['createdAt', 'entity', 'entityType', 'label', 'chain_id', 'confidence', 'remove', 'alert_id', 'handler_type', 'metadata'], data=[[created_date, entity, entityType, label_tag, chain_id, confidence, remove, alert_id, handler_type, metadata]])
#     df_labels = pd.concat([df_labels, df_temp_labels])



# df_labels['createdAt'] = pd.to_datetime(df_labels['createdAt'])  # Ensure the 'date' column is in datetime format
# df_sorted = df_labels.sort_values(by='createdAt')  # Sort DataFrame by 'date' column
# df_labels_deduplicated = df_sorted.drop_duplicates(subset=['entity', 'chain_id', 'hanlder_type'], keep='first')




# df_labels_deduplicated.reset_index(inplace=True)

# df_labels_deduplicated.to_csv(file_name)

In [6]:
def get_value(items: list, key: str):
    v = ''
    for item in items:
        if item.startswith(key):
            v = item.split('=')[1]
            break
    return v

In [10]:
# 'createdAt', 'id', 'label', 'source'
df_forta_labels['createdAt'] = pd.to_datetime(df_forta_labels['createdAt'])
df_forta_labels['chainId'] = df_forta_labels['source'].apply(lambda x: x['chainId'])
df_forta_labels['labelstr'] = df_forta_labels['label'].apply(lambda x: x['label'])
df_forta_labels['handlerType'] = df_forta_labels['labelstr'].apply(lambda x: x.split('/')[2] if '/' in x else x)
df_forta_labels['threatCategory'] = df_forta_labels['labelstr'].apply(lambda x: x.split('/')[1] if '/' in x else x)
df_forta_labels['labelstr'] = df_forta_labels['labelstr'].apply(lambda x: x.split('/')[0] if '/' in x else x)
df_forta_labels['entity'] = df_forta_labels['label'].apply(lambda x: x['entity'])
df_forta_labels['entityType'] = df_forta_labels['label'].apply(lambda x: x['entityType'])
df_forta_labels['remove'] = df_forta_labels['label'].apply(lambda x: x['remove'])
df_forta_labels['confidence'] = df_forta_labels['label'].apply(lambda x: x['confidence'])
df_forta_labels['metadata'] = df_forta_labels['label'].apply(lambda x: x['metadata'])
df_forta_labels['botVersion'] = df_forta_labels['label'].apply(lambda x: get_value(x['metadata'], 'bot_version'))
df_forta_labels['addressType'] = df_forta_labels['label'].apply(lambda x: get_value(x['metadata'], 'address_type'))
df_forta_labels['handlerType2'] = df_forta_labels['label'].apply(lambda x: get_value(x['metadata'], 'handler_type'))
df_forta_labels['handlerType3'] = df_forta_labels['label'].apply(lambda x: get_value(x['metadata'], 'logic'))
df_forta_labels['threatCategory2'] = df_forta_labels['label'].apply(lambda x: get_value(x['metadata'], 'threat_category'))

In [11]:
df_forta_labels.to_csv(file_name)

In [12]:
file_name

'ScamDetector_0x47c45816807d2eac30ba88745bf2778b61bc106bc76411b520a5289495c76db8_labels_20230614-20230622.csv'